# Task 4: Machine Learning & Statistical Modeling

## Objectives:
1. For each zipcode, fit a linear regression model that predicts total claims
2. Develop ML model that predicts optimal premium values
3. Report on important features

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [3]:
# Cell 2: Load and prepare data for machine learning
print("Loading and preparing data for machine learning...")

# Load the data
df = pd.read_csv('../data/raw/insurance_data.txt', delimiter='|', low_memory=False)

# Convert key columns to numeric
df['TotalPremium'] = pd.to_numeric(df['TotalPremium'], errors='coerce')
df['TotalClaims'] = pd.to_numeric(df['TotalClaims'], errors='coerce')
df['SumInsured'] = pd.to_numeric(df['SumInsured'], errors='coerce')

# Filter out zero premiums and create working dataset
df_ml = df[df['TotalPremium'] > 0].copy()

# Basic info
print(f"Dataset shape: {df_ml.shape}")
print(f"Rows: {df_ml.shape[0]:,}")
print(f"Columns: {df_ml.shape[1]}")
print()

# Check target variables
print("Target variables:")
print(f"TotalPremium mean: R{df_ml['TotalPremium'].mean():.2f}")
print(f"TotalClaims mean: R{df_ml['TotalClaims'].mean():.2f}")
print(f"SumInsured mean: R{df_ml['SumInsured'].mean():.2f}")
print()

# Check unique zipcodes
if 'PostalCode' in df_ml.columns:
    print(f"Unique zipcodes: {df_ml['PostalCode'].nunique():,}")
    print(f"Zipcodes with at least 50 policies: {(df_ml['PostalCode'].value_counts() >= 50).sum():,}")

Loading and preparing data for machine learning...
Dataset shape: (618176, 52)
Rows: 618,176
Columns: 52

Target variables:
TotalPremium mean: R100.20
TotalClaims mean: R100.41
SumInsured mean: R609826.86

Unique zipcodes: 858
Zipcodes with at least 50 policies: 746


In [4]:
# Cell 3: Prepare features for modeling
print("=" * 60)
print("PREPARING FEATURES FOR MACHINE LEARNING")
print("=" * 60)

# Select features for modeling
# Based on domain knowledge and data availability
potential_features = [
    # Policy/Client features
    'SumInsured',
    'ExcessSelected',  # Might be categorical
    'CoverType',
    'Product',
    'TermFrequency',
    
    # Client demographics
    'Province',
    'PostalCode',
    'Gender',
    'MaritalStatus',
    'AccountType',
    
    # Vehicle features
    'VehicleType',
    'make',
    'Model',
    'RegistrationYear',
    'Cylinders',
    'cubiccapacity',
    'bodytype',
    'NumberOfDoors',
    
    # Risk features
    'AlarmImmobiliser',
    'TrackingDevice',
    'NewVehicle',
]

# Check which features are available
available_features = [f for f in potential_features if f in df_ml.columns]
print(f"Available features: {len(available_features)}")
print("Features available for modeling:")
for i, feat in enumerate(available_features, 1):
    print(f"  {i:2d}. {feat}")
print()

# Check for missing values in these features
print("Missing values in selected features:")
missing_counts = {}
for feat in available_features:
    missing = df_ml[feat].isnull().sum()
    if missing > 0:
        missing_counts[feat] = missing
        print(f"  {feat:25} {missing:8,} ({missing/len(df_ml)*100:.1f}%)")

if not missing_counts:
    print("  No missing values in selected features!")

PREPARING FEATURES FOR MACHINE LEARNING
Available features: 21
Features available for modeling:
   1. SumInsured
   2. ExcessSelected
   3. CoverType
   4. Product
   5. TermFrequency
   6. Province
   7. PostalCode
   8. Gender
   9. MaritalStatus
  10. AccountType
  11. VehicleType
  12. make
  13. Model
  14. RegistrationYear
  15. Cylinders
  16. cubiccapacity
  17. bodytype
  18. NumberOfDoors
  19. AlarmImmobiliser
  20. TrackingDevice
  21. NewVehicle

Missing values in selected features:
  Gender                       4,621 (0.7%)
  MaritalStatus                5,071 (0.8%)
  AccountType                 30,734 (5.0%)
  VehicleType                    218 (0.0%)
  make                           218 (0.0%)
  Model                          218 (0.0%)
  Cylinders                      218 (0.0%)
  cubiccapacity                  218 (0.0%)
  bodytype                       218 (0.0%)
  NumberOfDoors                  218 (0.0%)
  NewVehicle                  60,634 (9.8%)


In [5]:
# Cell 4: Create final modeling dataset
print("=" * 60)
print("CREATING FINAL MODELING DATASET")
print("=" * 60)

# Create a copy for modeling
df_model = df_ml.copy()

# 1. Select features (start with a manageable set)
selected_features = [
    'SumInsured',           # Important: higher sum insured = higher risk
    'Province',             # Geographic risk
    'VehicleType',          # Type of vehicle
    'CoverType',            # Type of coverage
    'TermFrequency',        # Payment frequency
    'AlarmImmobiliser',     # Security feature
]

# Also include zipcode for grouping
if 'PostalCode' in df_model.columns:
    selected_features.append('PostalCode')

# Target variables
target_claims = 'TotalClaims'
target_premium = 'TotalPremium'

print(f"Selected features: {selected_features}")
print(f"Target 1: {target_claims} (for claims prediction)")
print(f"Target 2: {target_premium} (for premium prediction)")
print()

# Check data types
print("Data types of selected features:")
for feat in selected_features:
    if feat in df_model.columns:
        dtype = df_model[feat].dtype
        unique = df_model[feat].nunique() if dtype == 'object' else 'N/A'
        print(f"  {feat:20} {str(dtype):10} Unique: {unique}")

CREATING FINAL MODELING DATASET
Selected features: ['SumInsured', 'Province', 'VehicleType', 'CoverType', 'TermFrequency', 'AlarmImmobiliser', 'PostalCode']
Target 1: TotalClaims (for claims prediction)
Target 2: TotalPremium (for premium prediction)

Data types of selected features:
  SumInsured           float64    Unique: N/A
  Province             object     Unique: 9
  VehicleType          object     Unique: 5
  CoverType            object     Unique: 21
  TermFrequency        object     Unique: 2
  AlarmImmobiliser     object     Unique: 2
  PostalCode           int64      Unique: N/A


In [6]:
# Cell 5: Simple Linear Regression for TotalClaims
print("=" * 60)
print("OBJECTIVE 1: LINEAR REGRESSION FOR TOTAL CLAIMS")
print("=" * 60)

# For linear regression, we need to handle categorical variables
# Let's start with just numerical features for simplicity
print("Starting with numerical features only...")

# Select numerical features for initial model
numerical_features = ['SumInsured']  # Start simple

# Check correlation with TotalClaims
print("\n1. Correlation analysis:")
for feat in numerical_features:
    if feat in df_model.columns:
        corr = df_model[feat].corr(df_model['TotalClaims'])
        print(f"   {feat:20} Correlation with Claims: {corr:.4f}")

# Prepare data
X = df_model[numerical_features].fillna(df_model[numerical_features].mean())
y = df_model['TotalClaims']

# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\n2. Data split:")
print(f"   Training samples: {X_train.shape[0]:,}")
print(f"   Test samples: {X_test.shape[0]:,}")

# Train linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
y_pred = lr_model.predict(X_test)

# Evaluate model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n3. Model Performance:")
print(f"   Mean Squared Error (MSE): {mse:,.2f}")
print(f"   Root Mean Squared Error (RMSE): {rmse:,.2f}")
print(f"   Mean Absolute Error (MAE): {mae:,.2f}")
print(f"   R-squared (R²): {r2:.4f}")

print(f"\n4. Model Coefficients:")
for i, feat in enumerate(numerical_features):
    print(f"   {feat}: {lr_model.coef_[i]:.6f}")
print(f"   Intercept: {lr_model.intercept_:.2f}")

print(f"\n5. Interpretation:")
print(f"   • R² = {r2:.4f} means the model explains {r2*100:.1f}% of variance in claims")
print(f"   • RMSE = R{rmse:.2f} means average prediction error is R{rmse:.2f}")
if r2 < 0.1:
    print(f"   • WARNING: Very low R². Need more/better features.")

OBJECTIVE 1: LINEAR REGRESSION FOR TOTAL CLAIMS
Starting with numerical features only...

1. Correlation analysis:
   SumInsured           Correlation with Claims: -0.0063

2. Data split:
   Training samples: 494,540
   Test samples: 123,636

3. Model Performance:
   Mean Squared Error (MSE): 6,522,068.32
   Root Mean Squared Error (RMSE): 2,553.83
   Mean Absolute Error (MAE): 193.70
   R-squared (R²): 0.0001

4. Model Coefficients:
   SumInsured: -0.000012
   Intercept: 109.51

5. Interpretation:
   • R² = 0.0001 means the model explains 0.0% of variance in claims
   • RMSE = R2553.83 means average prediction error is R2553.83
   • WARNING: Very low R². Need more/better features.


In [7]:
# Cell 6: Adding Categorical Features with One-Hot Encoding
print("=" * 60)
print("ADDING CATEGORICAL FEATURES WITH ONE-HOT ENCODING")
print("=" * 60)

# Select categorical features to add
categorical_features = ['Province', 'VehicleType', 'CoverType']

# Create new feature set
all_features = numerical_features + categorical_features

print(f"Features for enhanced model: {all_features}")
print()

# Prepare data with one-hot encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Separate numerical and categorical columns
numerical_cols = numerical_features
categorical_cols = categorical_features

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ])

# Create pipeline with preprocessing and linear regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Split data
X = df_model[all_features]
y = df_model['TotalClaims']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
print("Training enhanced linear regression model...")
pipeline.fit(X_train, y_train)

# Evaluate
y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nEnhanced Model Performance:")
print(f"   R-squared (R²): {r2:.4f} (explains {r2*100:.1f}% of variance)")
print(f"   RMSE: R{rmse:.2f}")
print(f"   MAE: R{mae:.2f}")

# Compare with simple model
print(f"\nComparison with simple model:")
print(f"   Simple model R²: {r2:.4f}")
print(f"   Enhanced model R²: {r2:.4f}")
print(f"   Improvement: {(r2 - r2):.4f}")

# Get feature importance (for linear model, coefficients)
try:
    # Extract feature names after one-hot encoding
    cat_encoder = pipeline.named_steps['preprocessor'].named_transformers_['cat']
    cat_feature_names = cat_encoder.get_feature_names_out(categorical_cols)
    all_feature_names = numerical_cols + list(cat_feature_names)
    
    # Get coefficients
    coefficients = pipeline.named_steps['regressor'].coef_
    
    print(f"\nTop 10 most important features (by absolute coefficient):")
    feat_importance = pd.DataFrame({
        'Feature': all_feature_names,
        'Coefficient': coefficients
    })
    feat_importance['Abs_Coefficient'] = np.abs(feat_importance['Coefficient'])
    feat_importance = feat_importance.sort_values('Abs_Coefficient', ascending=False).head(10)
    
    for _, row in feat_importance.iterrows():
        print(f"   {row['Feature'][:40]:40} Coef: {row['Coefficient']:8.2f}")
        
except Exception as e:
    print(f"Could not extract feature importance: {e}")

ADDING CATEGORICAL FEATURES WITH ONE-HOT ENCODING
Features for enhanced model: ['SumInsured', 'Province', 'VehicleType', 'CoverType']

Training enhanced linear regression model...

Enhanced Model Performance:
   R-squared (R²): 0.0059 (explains 0.6% of variance)
   RMSE: R2546.34
   MAE: R209.93

Comparison with simple model:
   Simple model R²: 0.0059
   Enhanced model R²: 0.0059
   Improvement: 0.0000

Top 10 most important features (by absolute coefficient):
   CoverType_Standalone passenger liability Coef: -8900.04
   CoverType_Passenger Liability            Coef: -8895.80
   CoverType_Third Party Only               Coef: -1828.63
   CoverType_Own Damage                     Coef:  1642.52
   VehicleType_nan                          Coef:  1460.32
   CoverType_Income Protector               Coef:  1219.76
   CoverType_Trailer                        Coef:  1196.95
   CoverType_Windscreen                     Coef:  1188.67
   CoverType_Roadside Assistance            Coef:  1166.06
   

In [8]:
# Cell 7: Analyzing why linear regression fails
print("=" * 70)
print("ANALYSIS: WHY LINEAR REGRESSION FAILS FOR CLAIMS PREDICTION")
print("=" * 70)

print("\nKEY FINDING FROM DATA:")
print("1. Claim Distribution:")
print(f"   • Policies with ZERO claims: {(df_model['TotalClaims'] == 0).sum():,} (99.6%)")
print(f"   • Policies WITH claims: {(df_model['TotalClaims'] > 0).sum():,} (0.4%)")
print(f"   • Average claim amount (when claims occur): R{df_model[df_model['TotalClaims'] > 0]['TotalClaims'].mean():.2f}")
print()

print("2. Statistical Challenge:")
print("   • Linear regression assumes normal distribution")
print("   • Claims data is HIGHLY SKEWED: mostly zeros, few huge values")
print("   • This violates linear regression assumptions")
print()

print("3. Business Implication:")
print("   • Traditional pricing models may fail for this portfolio")
print("   • Need specialized models for rare, catastrophic events")
print("   • Consider: Zero-inflated models, Poisson regression, etc.")
print()

print("RECOMMENDED APPROACHES:")
print("1. Two-stage modeling:")
print("   • Stage 1: Predict IF a claim will occur (classification)")
print("   • Stage 2: Predict HOW MUCH claim will be (regression on claims>0)")
print()
print("2. Alternative models:")
print("   • Random Forest / Gradient Boosting (handle skewed data better)")
print("   • Zero-inflated Poisson regression")
print("   • Quantile regression (predict worst-case scenarios)")

ANALYSIS: WHY LINEAR REGRESSION FAILS FOR CLAIMS PREDICTION

KEY FINDING FROM DATA:
1. Claim Distribution:
   • Policies with ZERO claims: 615,533 (99.6%)
   • Policies WITH claims: 2,641 (0.4%)
   • Average claim amount (when claims occur): R23510.32

2. Statistical Challenge:
   • Linear regression assumes normal distribution
   • Claims data is HIGHLY SKEWED: mostly zeros, few huge values
   • This violates linear regression assumptions

3. Business Implication:
   • Traditional pricing models may fail for this portfolio
   • Need specialized models for rare, catastrophic events
   • Consider: Zero-inflated models, Poisson regression, etc.

RECOMMENDED APPROACHES:
1. Two-stage modeling:
   • Stage 1: Predict IF a claim will occur (classification)
   • Stage 2: Predict HOW MUCH claim will be (regression on claims>0)

2. Alternative models:
   • Random Forest / Gradient Boosting (handle skewed data better)
   • Zero-inflated Poisson regression
   • Quantile regression (predict worst-c

In [9]:
# Cell 8: Random Forest for Claims Prediction
print("=" * 60)
print("RANDOM FOREST REGRESSOR (Handles skewed data better)")
print("=" * 60)

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# Prepare data for Random Forest (handles categorical natively)
# Let's encode categorical variables
df_rf = df_model.copy()

# Encode categorical variables
label_encoders = {}
categorical_cols = ['Province', 'VehicleType', 'CoverType', 'TermFrequency', 'AlarmImmobiliser']

for col in categorical_cols:
    if col in df_rf.columns:
        le = LabelEncoder()
        # Handle NaN values
        df_rf[col] = df_rf[col].fillna('Missing')
        df_rf[col] = le.fit_transform(df_rf[col].astype(str))
        label_encoders[col] = le

# Features for Random Forest
rf_features = ['SumInsured'] + categorical_cols
if 'PostalCode' in df_rf.columns:
    rf_features.append('PostalCode')

print(f"Features for Random Forest: {rf_features}")
print()

# Prepare X and y
X = df_rf[rf_features].fillna(0)
y = df_rf['TotalClaims']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=(y > 0))

print(f"Training samples: {X_train.shape[0]:,}")
print(f"Test samples: {X_test.shape[0]:,}")
print(f"Claims in training set: {(y_train > 0).sum():,} ({((y_train > 0).sum()/len(y_train))*100:.2f}%)")
print(f"Claims in test set: {(y_test > 0).sum():,} ({((y_test > 0).sum()/len(y_test))*100:.2f}%)")
print()

# Train Random Forest (with limited trees for speed)
print("Training Random Forest (this may take a minute)...")
rf_model = RandomForestRegressor(
    n_estimators=100,  # Reduced for speed
    max_depth=10,
    min_samples_split=100,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred_rf = rf_model.predict(X_test)

# Calculate metrics
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"\nRandom Forest Performance:")
print(f"   R-squared (R²): {r2_rf:.4f} (explains {r2_rf*100:.1f}% of variance)")
print(f"   RMSE: R{rmse_rf:.2f}")
print(f"   MAE: R{mae_rf:.2f}")
print()

# Feature importance
print("Top 10 Feature Importances:")
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': rf_features,
    'Importance': importances
}).sort_values('Importance', ascending=False)

for i, row in feature_importance_df.head(10).iterrows():
    print(f"   {row['Feature']:25} {row['Importance']:.4f}")
    

RANDOM FOREST REGRESSOR (Handles skewed data better)
Features for Random Forest: ['SumInsured', 'Province', 'VehicleType', 'CoverType', 'TermFrequency', 'AlarmImmobiliser', 'PostalCode']

Training samples: 494,540
Test samples: 123,636
Claims in training set: 2,113 (0.43%)
Claims in test set: 528 (0.43%)

Training Random Forest (this may take a minute)...

Random Forest Performance:
   R-squared (R²): 0.0234 (explains 2.3% of variance)
   RMSE: R2896.79
   MAE: R190.77

Top 10 Feature Importances:
   SumInsured                0.5342
   PostalCode                0.2814
   CoverType                 0.1509
   VehicleType               0.0212
   Province                  0.0123
   AlarmImmobiliser          0.0000
   TermFrequency             0.0000


In [10]:
# Cell 9: Linear Regression per Zipcode (for top 5 zipcodes)
print("=" * 70)
print("OBJECTIVE 1: LINEAR REGRESSION FOR EACH ZIPCODE")
print("=" * 70)
print("Demonstrating methodology for top 5 zipcodes by policy count")
print()

# Get top 5 zipcodes
top_zipcodes = df_model['PostalCode'].value_counts().head(5).index

results = []

for zipcode in top_zipcodes:
    print(f"\n{'='*40}")
    print(f"ZIPCODE: {zipcode}")
    print(f"{'='*40}")
    
    # Filter data for this zipcode
    zip_data = df_model[df_model['PostalCode'] == zipcode].copy()
    
    if len(zip_data) < 50:
        print(f"   Skipping - only {len(zip_data)} policies")
        continue
    
    print(f"   Policies: {len(zip_data):,}")
    print(f"   Policies with claims: {(zip_data['TotalClaims'] > 0).sum():,} ({(zip_data['TotalClaims'] > 0).sum()/len(zip_data)*100:.1f}%)")
    
    # Simple linear regression: SumInsured -> TotalClaims
    X_zip = zip_data[['SumInsured']].fillna(zip_data['SumInsured'].mean())
    y_zip = zip_data['TotalClaims']
    
    if len(X_zip) > 10:
        # Train model
        lr_zip = LinearRegression()
        lr_zip.fit(X_zip, y_zip)
        
        # Predict
        y_pred_zip = lr_zip.predict(X_zip)
        
        # Calculate R²
        r2_zip = r2_score(y_zip, y_pred_zip)
        
        print(f"   Linear Regression R²: {r2_zip:.4f}")
        print(f"   Coefficient (SumInsured): {lr_zip.coef_[0]:.8f}")
        print(f"   Intercept: {lr_zip.intercept_:.2f}")
        
        results.append({
            'Zipcode': zipcode,
            'Policies': len(zip_data),
            'ClaimsPct': (zip_data['TotalClaims'] > 0).sum()/len(zip_data)*100,
            'R2': r2_zip,
            'Coefficient': lr_zip.coef_[0],
            'Intercept': lr_zip.intercept_
        })
    else:
        print("   Insufficient data for regression")

print(f"\n{'='*70}")
print("SUMMARY: Linear Regression per Zipcode")
print("="*70)

if results:
    results_df = pd.DataFrame(results)
    print(results_df.to_string(index=False))
    
    print(f"\nKey Insights:")
    print(f"1. All zipcodes have very low claim frequency (< 0.5%)")
    print(f"2. R² values are very low (near zero)")
    print(f"3. SumInsured has minimal predictive power for claims")
    print(f"4. Need more sophisticated modeling approach")
else:
    print("No valid results - insufficient data in individual zipcodes")

OBJECTIVE 1: LINEAR REGRESSION FOR EACH ZIPCODE
Demonstrating methodology for top 5 zipcodes by policy count


ZIPCODE: 2000
   Policies: 90,934
   Policies with claims: 471 (0.5%)
   Linear Regression R²: 0.0001
   Coefficient (SumInsured): -0.00001835
   Intercept: 110.24

ZIPCODE: 122
   Policies: 27,898
   Policies with claims: 190 (0.7%)
   Linear Regression R²: 0.0001
   Coefficient (SumInsured): -0.00001897
   Intercept: 130.29

ZIPCODE: 299
   Policies: 16,731
   Policies with claims: 65 (0.4%)
   Linear Regression R²: 0.0001
   Coefficient (SumInsured): -0.00000852
   Intercept: 54.53

ZIPCODE: 7784
   Policies: 13,560
   Policies with claims: 45 (0.3%)
   Linear Regression R²: 0.0000
   Coefficient (SumInsured): -0.00001750
   Intercept: 132.47

ZIPCODE: 7405
   Policies: 10,731
   Policies with claims: 29 (0.3%)
   Linear Regression R²: 0.0002
   Coefficient (SumInsured): -0.00001118
   Intercept: 64.41

SUMMARY: Linear Regression per Zipcode
 Zipcode  Policies  ClaimsPct   

In [14]:
# Cell 10 (FIXED COMPLETE): Prepare data for premium prediction
print("=" * 70)
print("OBJECTIVE 2: MACHINE LEARNING FOR OPTIMAL PREMIUM PREDICTION")
print("=" * 70)

# Calculate LossRatio if not exists
if 'LossRatio' not in df_model.columns:
    df_model['LossRatio'] = df_model['TotalClaims'] / df_model['TotalPremium']
    df_model['LossRatio'] = df_model['LossRatio'].replace([np.inf, -np.inf], np.nan)

# Identify profitable policies
df_model['Profitable'] = df_model['LossRatio'] < 1
profitable_pct = df_model['Profitable'].mean() * 100

print(f"Profitable policies: {profitable_pct:.1f}%")
print()

# Prepare data for premium prediction
df_premium = df_model.copy()

# Encode categorical variables
categorical_cols = ['Province', 'VehicleType', 'CoverType', 'TermFrequency', 'AlarmImmobiliser']
for col in categorical_cols:
    if col in df_premium.columns:
        le = LabelEncoder()
        df_premium[col] = df_premium[col].fillna('Missing')
        df_premium[col] = le.fit_transform(df_premium[col].astype(str))

# Features for premium prediction
premium_features = [
    'SumInsured',
    'Province',
    'VehicleType', 
    'CoverType',
    'TermFrequency',
    'AlarmImmobiliser',
    'PostalCode'
]

# Filter to only profitable policies for training
df_profitable = df_premium[df_premium['Profitable']].copy()
print(f"Profitable policies for training: {len(df_profitable):,}")

# Define X and y
X_premium = df_profitable[premium_features].fillna(0)
y_premium = df_profitable['TotalPremium']

# Split data
from sklearn.model_selection import train_test_split
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(
    X_premium, y_premium, test_size=0.2, random_state=42
)

print(f"Training data: {X_train_p.shape[0]:,} samples")
print(f"Test data: {X_test_p.shape[0]:,} samples")
print(f"Average premium in training: R{y_train_p.mean():.2f}")

OBJECTIVE 2: MACHINE LEARNING FOR OPTIMAL PREMIUM PREDICTION
Profitable policies: 99.6%

Profitable policies for training: 615,551
Training data: 492,440 samples
Test data: 123,111 samples
Average premium in training: R98.56


In [15]:
# Cell 11 (SIMPLIFIED): Gradient Boosting for Premium Prediction
print("=" * 60)
print("GRADIENT BOOSTING FOR PREMIUM PREDICTION")
print("=" * 60)

# Make sure X_train_p exists
if 'X_train_p' not in locals():
    print("ERROR: X_train_p not defined. Run cell 10 first.")
else:
    from sklearn.ensemble import GradientBoostingRegressor
    
    print("Training Gradient Boosting Regressor...")
    
    # Train with fewer estimators for speed
    gb_model = GradientBoostingRegressor(
        n_estimators=50,  # Reduced for speed
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    )
    
    gb_model.fit(X_train_p, y_train_p)
    
    # Predict
    y_pred_gb = gb_model.predict(X_test_p)
    
    # Evaluate
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    mse_gb = mean_squared_error(y_test_p, y_pred_gb)
    rmse_gb = np.sqrt(mse_gb)
    mae_gb = mean_absolute_error(y_test_p, y_pred_gb)
    r2_gb = r2_score(y_test_p, y_pred_gb)
    
    print(f"\nModel Performance:")
    print(f"   R-squared (R²): {r2_gb:.4f} (explains {r2_gb*100:.1f}% of variance)")
    print(f"   RMSE: R{rmse_gb:.2f}")
    print(f"   MAE: R{mae_gb:.2f}")

GRADIENT BOOSTING FOR PREMIUM PREDICTION
Training Gradient Boosting Regressor...

Model Performance:
   R-squared (R²): 0.4252 (explains 42.5% of variance)
   RMSE: R250.85
   MAE: R19.52


In [16]:
# Cell 12: Apply model to ALL policies to suggest optimal premiums
print("=" * 70)
print("SUGGESTING OPTIMAL PREMIUMS FOR ALL POLICIES")
print("=" * 70)

# Prepare all data for prediction
X_all = df_premium[premium_features].fillna(0)

# Predict optimal premiums using our model
print("Predicting optimal premiums for all policies...")
optimal_premiums = gb_model.predict(X_all)

# Add predictions to dataframe
df_premium['OptimalPremium'] = optimal_premiums

# Compare current vs optimal
df_premium['PremiumDifference'] = df_premium['OptimalPremium'] - df_premium['TotalPremium']
df_premium['PremiumChangePct'] = (df_premium['PremiumDifference'] / df_premium['TotalPremium']) * 100

print(f"\nComparison: Current vs Optimal Premiums")
print(f"Current average premium: R{df_premium['TotalPremium'].mean():.2f}")
print(f"Optimal average premium: R{df_premium['OptimalPremium'].mean():.2f}")
print(f"Average change: R{df_premium['PremiumDifference'].mean():.2f}")
print(f"Average % change: {df_premium['PremiumChangePct'].mean():.1f}%")
print()

# Analyze by profitability
print("Premium Adjustment Analysis:")
print("-" * 50)

for profitable in [True, False]:
    subset = df_premium[df_premium['Profitable'] == profitable]
    label = "Profitable" if profitable else "Unprofitable"
    
    avg_current = subset['TotalPremium'].mean()
    avg_optimal = subset['OptimalPremium'].mean()
    avg_change = subset['PremiumDifference'].mean()
    avg_pct_change = subset['PremiumChangePct'].mean()
    
    print(f"\n{label} Policies ({len(subset):,} policies):")
    print(f"   Current premium: R{avg_current:.2f}")
    print(f"   Optimal premium: R{avg_optimal:.2f}")
    print(f"   Suggested change: R{avg_change:+.2f} ({avg_pct_change:+.1f}%)")
    
    if profitable and avg_change < 0:
        print(f"   ✓ Suggests REDUCING premium (attract more customers)")
    elif not profitable and avg_change > 0:
        print(f"   ✓ Suggests INCREASING premium (improve profitability)")

SUGGESTING OPTIMAL PREMIUMS FOR ALL POLICIES
Predicting optimal premiums for all policies...

Comparison: Current vs Optimal Premiums
Current average premium: R100.20
Optimal average premium: R99.61
Average change: R-0.59
Average % change: 2231.2%

Premium Adjustment Analysis:
--------------------------------------------------

Profitable Policies (615,551 policies):
   Current premium: R98.84
   Optimal premium: R98.68
   Suggested change: R-0.16 (+2240.7%)
   ✓ Suggests REDUCING premium (attract more customers)

Unprofitable Policies (2,625 policies):
   Current premium: R419.41
   Optimal premium: R317.35
   Suggested change: R-102.06 (+1.5%)


In [17]:
# Cell 13: Final Recommendations and Business Impact
print("=" * 70)
print("FINAL RECOMMENDATIONS: OPTIMAL PRICING STRATEGY")
print("=" * 70)

print("\n1. KEY FINDINGS FROM MACHINE LEARNING:")
print("   • Claims are RARE (0.4%) but CATASTROPHIC when they occur")
print("   • Traditional linear models fail (R² < 0.01)")
print("   • Random Forest explains 2.3% of claim variance")
print("   • Most important factors: SumInsured, PostalCode, CoverType")
print()

print("2. OPTIMAL PRICING MODEL RESULTS:")
print(f"   • Gradient Boosting R²: {r2_gb:.4f} ({r2_gb*100:.1f}% variance explained)")
print("   • Model trained on profitable policies only")
print("   • Can suggest premium adjustments for all policies")
print()

print("3. PREMIUM ADJUSTMENT STRATEGY:")
print("   For PROFITABLE policies (LossRatio < 1):")
print("   • Consider SMALL premium REDUCTIONS")
print("   • Attract more customers in low-risk segments")
print("   • Gain market share while maintaining profitability")
print()
print("   For UNPROFITABLE policies (LossRatio ≥ 1):")
print("   • Need premium INCREASES")
print("   • Or reconsider risk appetite for these segments")
print("   • Focus on zipcodes/vehicle types with worst loss ratios")
print()

print("4. IMPLEMENTATION RECOMMENDATIONS:")
print("   a. PHASED APPROACH:")
print("      • Start with 5-10% premium adjustments")
print("      • Monitor impact on portfolio profitability")
print("      • Adjust based on actual results")
print()
print("   b. SEGMENT-SPECIFIC STRATEGY:")
print("      • Different adjustments by province, zipcode, vehicle type")
print("      • Use model feature importances to guide strategy")
print()
print("   c. CONTINUOUS MONITORING:")
print("      • Update models quarterly with new data")
print("      • Track actual vs predicted loss ratios")
print("      • Refine models based on performance")
print()

print("5. EXPECTED BUSINESS IMPACT:")
print("   • Improved portfolio profitability")
print("   • More competitive pricing for low-risk segments")
print("   • Better risk selection and pricing accuracy")
print("   • Data-driven decision making")

FINAL RECOMMENDATIONS: OPTIMAL PRICING STRATEGY

1. KEY FINDINGS FROM MACHINE LEARNING:
   • Claims are RARE (0.4%) but CATASTROPHIC when they occur
   • Traditional linear models fail (R² < 0.01)
   • Random Forest explains 2.3% of claim variance
   • Most important factors: SumInsured, PostalCode, CoverType

2. OPTIMAL PRICING MODEL RESULTS:
   • Gradient Boosting R²: 0.4252 (42.5% variance explained)
   • Model trained on profitable policies only
   • Can suggest premium adjustments for all policies

3. PREMIUM ADJUSTMENT STRATEGY:
   For PROFITABLE policies (LossRatio < 1):
   • Consider SMALL premium REDUCTIONS
   • Attract more customers in low-risk segments
   • Gain market share while maintaining profitability

   For UNPROFITABLE policies (LossRatio ≥ 1):
   • Need premium INCREASES
   • Or reconsider risk appetite for these segments
   • Focus on zipcodes/vehicle types with worst loss ratios

4. IMPLEMENTATION RECOMMENDATIONS:
   a. PHASED APPROACH:
      • Start with 5-10% p

In [18]:
# Cell 14: Save Model Results and Predictions
print("=" * 60)
print("SAVING MODEL RESULTS AND PREDICTIONS")
print("=" * 60)

# Save predictions to CSV for business use
output_cols = [
    'UnderwrittenCoverID', 'PolicyID', 'PostalCode', 'Province',
    'VehicleType', 'CoverType', 'SumInsured',
    'TotalPremium', 'TotalClaims', 'LossRatio',
    'OptimalPremium', 'PremiumDifference', 'PremiumChangePct', 'Profitable'
]

# Only save columns that exist
existing_cols = [col for col in output_cols if col in df_premium.columns]

# Create reports directory if not exists
import os
os.makedirs('../data/processed', exist_ok=True)

df_premium[existing_cols].to_csv('../data/processed/optimal_premium_predictions.csv', index=False)

print(f"Predictions saved to: data/processed/optimal_premium_predictions.csv")
print(f"Rows: {len(df_premium):,}")
print(f"Columns: {len(existing_cols)}")
print("\nFile includes:")
for col in existing_cols:
    print(f"  • {col}")

SAVING MODEL RESULTS AND PREDICTIONS
Predictions saved to: data/processed/optimal_premium_predictions.csv
Rows: 618,176
Columns: 14

File includes:
  • UnderwrittenCoverID
  • PolicyID
  • PostalCode
  • Province
  • VehicleType
  • CoverType
  • SumInsured
  • TotalPremium
  • TotalClaims
  • LossRatio
  • OptimalPremium
  • PremiumDifference
  • PremiumChangePct
  • Profitable
